# Практика
Используемые библиотеки

In [2]:
import pandas as pd
import numpy as np
import requests
import io
import re
import math


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [64]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
display(df)


,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [78]:
# Ваше решение
df["Значения"] = df["Норматив"].replace(to_replace="[не более в пределах 6 7 2 -]", value='', regex=True)
df1 = df.replace("б/цвета", 0)
df1["Значения"] = df["Значения"].replace(to_replace="[,]", value='.', regex=True)
df1['Результат анализа'] = df1['Результат анализа'].astype(float)
df1['Значения'] = df1['Значения'].astype(float)
comparison_column = np.where(df1["Значения"] < df1["Результат анализа"], ["Показатели превышены"], ["Показатели  в норме"])
comparison_column
df1["Итого"] = comparison_column
df1


,Показатель,Единица измерений,Результат анализа,Норматив,Значения,Итого
0,pH,единицы pH,8.400,в пределах 6-9,9.0,Показатели в норме
1,Запах,баллы,1.000,не более 2-3,3.0,Показатели в норме
2,Цветность,градусы,0.000,не более 30,30.0,Показатели в норме
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,10.0,Показатели в норме
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",1.5,Показатели в норме
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",3.3,Показатели в норме
6,Нитраты (по NO3),мг/дм3,24.000,не более 45,45.0,Показатели в норме
7,Фосфаты (P),мг/дм3,0.360,"не более 3,5",3.5,Показатели в норме
8,Хлориды (Cl),мг/дм3,200.000,не более 350,350.0,Показатели в норме
9,Сульфаты (SO4),мг/дм3,189.500,не более 500,500.0,Показатели в норме


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [164]:
# Ваше решение
box = ["o", "o", "o", "o", "o", "a", "a", "a", "a"]
N = 1
list_var = []
for i in range(0, 10000):
    np.random.shuffle(box)
    list_var.append(box[:])
list_var
df = pd.DataFrame(list_var)
df1 = df.loc[(df[0] == "o") & (df[1] == "o") & (df[2] == "o")]
result = df1.count()/df.count()
result


0    0.1165
1    0.1165
2    0.1165
3    0.1165
4    0.1165
5    0.1165
6    0.1165
7    0.1165
8    0.1165
dtype: float64

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [5]:
# Ваше решение